In [31]:
def classCount(rows):
    counts={}
    for row in rows:
        label = row[-1]
        if label not in  counts:
            counts[label] = 0
        else:
            counts[label]+= 1
    return counts

def gini(rows):
    counts = classCount(rows)
    impurity = 1
    for label in counts:
        prob = counts[label]/float(len(rows))
        impurity -= prob**2
    return impurity


def infogain(left,right,current_uncertainity):
    prob = float(len(left))/(len(left)+len(right))
    return current_uncertainity - prob*gini(left)-(1-prob)*gini(right)

def partition(rows,question):
    true_rows,false_rows = [],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows,false_rows

class Question:
    def __init__(self,column,value):
        self.column = column
        self.value = value
    def match(self,example):
        val = example[self.column]
        #print(val)
        if isinstance(val, int) or isinstance(val, float):
            return val >= self.value
        else:
            return val == self.value

def find_best_split(rows):
    max_gain = 0
    best_question = None
    current_uncertainity = gini(rows)
    n_features = len(rows[0])-1
    
    for col in range(n_features):
        values = set([row[col] for row in rows])
        
        for val in values:
            question = Question(col,val)
            true_rows,false_rows = partition(rows,question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = infogain(true_rows,false_rows,current_uncertainity)
            if gain >= max_gain:
                max_gain , best_question = gain,question
     
    
    return max_gain,best_question

class Leaf:

    def __init__(self, rows):
        self.predictions = classCount(rows)

class Decision_Node:

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

def buildTree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows,false_rows = partition(rows,question)
    true_branch = buildTree(true_rows)
    false_branch = buildTree(false_rows)
    return Decision_Node(question, true_branch, false_branch)

def classify(row,node):
    if isinstance(node,Leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row,node.true_branch)
    else:
        return classify(row,node.false_branch)

In [32]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]


my_tree = buildTree(training_data)

testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], classify(row, my_tree)))

Actual: Apple. Predicted: {'Apple': 1, 'Lemon': 0}
Actual: Apple. Predicted: {'Apple': 1, 'Lemon': 0}
Actual: Grape. Predicted: {'Grape': 1}
Actual: Grape. Predicted: {'Grape': 1}
Actual: Lemon. Predicted: {'Apple': 1, 'Lemon': 0}
